In [1]:
from keras.applications import vgg16


Using TensorFlow backend.


In [2]:
from keras.layers import  Dense 

In [3]:
model = vgg16.VGG16(weights= 'imagenet' , include_top = False ,input_shape= (224,224,3))






In [4]:
m=0
for i in model.layers:
    m = m+1
print(m)


19


In [5]:
model.layers

In [6]:
model.layers[0].trainable = False

In [7]:
model.layers[0]

In [8]:
model.layers[0].trainable

False

In [9]:
model.layers[1].trainable

True

In [10]:
for i in model.layers:
    i.trainable = False
for (j,layer) in enumerate (model.layers):
     print(str(i)+" " + layer.__class__.__name__,layer.trainable)

<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> InputLayer False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> MaxPooling2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> MaxPooling2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> MaxPooling2D False
<keras.layers.pooling.MaxPooling2D object at 0x000002F025C37B08> Conv2D False
<keras.layers.pooling.MaxPooling2D object 

In [11]:
model.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [12]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
from keras.layers import GlobalAveragePooling2D,Conv2D

In [14]:
top_model = model.output
#top_model = Conv2D()
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(1024,activation = 'relu')(top_model)

In [15]:
top_model = Dense(1024 , activation = 'relu')(top_model)
top_model = Dense(512 , activation = 'relu')(top_model)
top_model  = Dense(256 , activation = 'relu')(top_model)
top_model = Dense(3,activation = 'softmax')(top_model)

In [16]:
from keras.models import Sequential ,Model
from keras.layers import Flatten,MaxPooling2D ,Conv2D

In [17]:
model = Model(inputs = model.input , outputs = top_model)

In [18]:
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                                 rescale = 1./255,
                                 rotation_range = 45,
                                 width_shift_range = 0.3,
                                 height_shift_range = 0.3,
                                 horizontal_flip = True,
                                 fill_mode = 'nearest')
validation_datgen = ImageDataGenerator(rescale = 1./255)
batch_size = 32
train_generator = train_datgen.flow_from_directory(
                   'facetrain'
                    target_size = (224,224,3)
                    batch_size = batch_size
                    class_mode = 'categorical')
validation_generator

In [20]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 45,
                                   width_shift_range = 0.3,
                                   zoom_range=0.2,
                                   shear_range = 0.2,
                                   height_shift_range = 0.3,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest') 
validation_datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 32 
train_generator = train_datagen.flow_from_directory('facetrain',
                                                   target_size = (224,224),
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory('New folder (2)',
                                                             target_size = (224,224),
                                                             batch_size = batch_size,
                                                             class_mode = 'categorical')

Found 337 images belonging to 3 classes.
Found 17 images belonging to 3 classes.


In [21]:

from keras.callbacks import ModelCheckpoint , EarlyStopping

In [22]:
Checkpoint =ModelCheckpoint('face_recognition' ,
                            monitor ='val_loss',
                            verbose=1,
                            mode = 'min',
                            save_best_only=True)
Stopping = EarlyStopping( monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True,)
callback = [Checkpoint , Stopping]
model.compile(optimizer = 'adam' , loss ='categorical_crossentropy' , metrics=['accuracy'])
epoch = 3
batch_size = 16
history = model.fit_generator(train_generator,
                             steps_per_epoch = 337//batch_size, 
                             epochs= epoch,
                             callbacks = callback,
                             validation_data = validation_generator,
                             validation_steps = 17//batch_size)
                    

Epoch 1/3
21/21 [==============================] - 280s 13s/step - loss: 0.9586 - accuracy: 0.5171 - val_loss: 1.6534 - val_accuracy: 0.3529

Epoch 00001: val_loss improved from inf to 1.65340, saving model to face_recognition
Epoch 2/3
21/21 [==============================] - 275s 13s/step - loss: 0.5302 - accuracy: 0.7913 - val_loss: 2.9997 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 1.65340
Epoch 3/3
21/21 [==============================] - 281s 13s/step - loss: 0.2980 - accuracy: 0.8782 - val_loss: 3.0078 - val_accuracy: 0.0588

Epoch 00003: val_loss did not improve from 1.65340
